In [10]:
import pandas as pd
import numpy as np
import os

import regex as re
import altair as alt

from datetime import datetime
from datetime import timedelta

In [11]:
"The visualizations that Vicens made were edited by Eric to create a mosre consistent color scheme throughout the paper.\
this file is a copy of the work Vicens did, where Eric altered the visualizations used in the final paper."

'The visualizations that Vicens made were edited by Eric to create a mosre consistent color scheme throughout the paper.this file is a copy of the work Vicens did, where Eric altered the visualizations used in the final paper.'

In [12]:
# os.chdir('w2020-data599-capstone-projects-statistics-canada-covid-19')
# os.chdir('../../..')
os.getcwd()

'C:\\Users\\ebaxt\\Documents\\Masters\\capstone\\w2020-data599-capstone-projects-statistics-canada-covid-19'

In [13]:
'/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/code/CEWS_v1_0-eng/'
'/Users/vicenspaneque/MDS/599 Capstone Project/w2020-data599-capstone-projects-statistics-canada-covid-19/'

# Databases loading
cews = pd.read_csv("CEWS_updated.csv")

## This dataset is taken from StatCan and available in our repo (https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1410037701).
re_counts = pd.read_csv("more_data/RuralEmploymentIndustry.csv")
ue_counts = pd.read_csv("more_data/UrbanEmploymentIndustry.csv")

In [14]:
# Specify color schemes

# Alphabetical province list
provinces = sorted(list(cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'All industries'")["Province"].unique()))
# Add canadawide color
provinces = provinces + ["Canada"]
colors = ["#1f77b4", "#aec7e8", "#ff7f0e", "#2ca02c", "#d62728", "#ff9896", "#9467bd", "#8c564b", "#7f7f7f", "#c7c7c7", "#bcbd22", "#17becf", "#98df8a"]
colors = colors + ["red"]
provColors = pd.DataFrame(list(zip(provinces, colors)), columns = ["Province", "Color"]) 

industries = sorted(list(cews.query("GeoAggregation == 'Canada' and IndustryAggregation == 'Level 1'")["Industry"].unique()))
colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#000000']
indColors = pd.DataFrame(list(zip(industries, colors)), columns = ["Industry", "Color"]) 

In [15]:
# I need to define a function which pulls out the desired provinces and colors as lists to be fed into altair
# For some reason there is no native way to match a color to a region, without including all possible values in the legend

def getProvCols(provinces):
    provinces = sorted(provinces)
    cols = provColors[provColors["Province"].isin(provinces)]
    cols = list(cols["Color"])
    return cols

def getIndCols(industries):
    industries = sorted(industries)
    cols = indColors[indColors["Industry"].isin(industries)]
    cols = list(cols["Color"])
    return cols

In [16]:
# Changing data types
cews.Period = pd.to_datetime(cews.Period, infer_datetime_format=True)

# Indicating intereset varaibles for analysis
num_cols = ['BusinessLocations', 'Subsidy', 'SupportedEmployees', 'RehiredEmployees']
cum_cols = ['BusinessLocations Cumulative', 'Subsidy Cumulative', 'SupportedEmployees Cumulative', 'RehiredEmployees Cumulative']
change_cols = ['BusinessLocations Change', 'Subsidy Change', 'SupportedEmployees Change', 'RehiredEmployees Change']
cumchange_cols = ['BusinessLocations Cumulative Change', 'Subsidy Cumulative Change', 'SupportedEmployees Cumulative Change', 'RehiredEmployees Cumulative Change']
provinces = ['10-rural', '11-rural', '12-rural', '13-rural', '24-rural', '35-rural', '46-rural', '47-rural', '48-rural', '59-rural']
provinces_ur = ['10-urban', '11-urban', '12-urban', '13-urban', '24-urban', '35-urban', '46-urban', '47-urban', '48-urban', '59-urban']
territories = ['60-rural', '61-rural', '62-rural']
industries = ['Construction', 'Manufacturing', 'Accommodation and food services', 'Mining, quarrying, and oil and gas extraction']

cews.head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
0,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,11,"Agriculture, forestry, fishing and hunting",30.0,823000.0,362.0,0.0,Province,Level 1,10.0,Newfoundland and Labrador,5605.0
1,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,111,Crop production,10.0,NaN,90.0,0.0,Province,Level 2,10.0,Newfoundland and Labrador,NaN
2,2020-03-15,10,Newfoundland and Labrador,Not applicable,Not applicable,112,Animal production and aquaculture,10.0,NaN,NaN,0.0,Province,Level 2,10.0,Newfoundland and Labrador,NaN


# CEWS Rural Analysis
# A top-down industry lens approach

## Canada totals

### CEWS evolution and average number of rehired employees and supported employees and businesses by claim period

In [17]:
cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'").head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
71050,2020-03-15,TOTAL,Canada,Not applicable,Not applicable,TOTAL,All Industries,294070.0,7.459996e+09,3489214.0,30520.0,Canada,All industries,NaN,NaN,17230040.0
148158,2020-04-12,TOTAL,Canada,Not applicable,Not applicable,TOTAL,All Industries,336595.0,9.111729e+09,3743506.0,38945.0,Canada,All industries,NaN,NaN,17230040.0
225118,2020-05-10,TOTAL,Canada,Not applicable,Not applicable,TOTAL,All Industries,331865.0,9.647710e+09,4011166.0,44420.0,Canada,All industries,NaN,NaN,17230040.0


In [18]:
fig1 = alt.Chart(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('sum(Subsidy):Q', format=('$,.0f'))
        ).properties(title={'text': 'Total value of subsidies approved'}
                    )#.configure_axis(domain=False, grid=False, ticks=False
                             #   ).configure_view(stroke="transparent")

In [19]:
fig2 = alt.Chart(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(Subsidy):Q', format=('$,.0f'))
        ).properties(title={'text': 'Average value of subsidies approved per claim period'}
                    )#.configure_axis(domain=False, grid=False, ticks=False
                      #          ).configure_view(stroke="transparent")

In [20]:
fig3 = alt.Chart(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(SupportedEmployees):Q', format=(',.0f'))
        ).properties(title={'text': 'CEWS average employees supported per claim period'}
                    )#.configure_axis(domain=False, grid=False, ticks=False
                                #).configure_view(stroke="transparent")

In [21]:
alt.Chart(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(RehiredEmployees):Q', format=(',.0f'))
        ).properties(title={'text': 'Average rehired employees per claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [22]:
fig4 = alt.Chart(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(BusinessLocations):Q', format=(',.0f'))
        ).properties(title={'text': 'Average businesses supported per claim period'}
                    )#.configure_axis(domain=False, grid=False, ticks=False
                               # ).configure_view(stroke="transparent")

In [23]:
#(fig1 | fig2) & 
((fig1 | fig2) & (fig3 | fig4)).configure_axis(domain=False, grid=False, ticks=False).configure_view(stroke="transparent").\
    properties(title = "High level overview of the CEWS program, March 15 through Oct 24, 2020")


alt.VConcatChart(...)

In [24]:
alt.Chart(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")).mark_bar(size=40).encode(
    x=alt.X('Period:T', title='Date', scale=alt.Scale(domain=["2020-03-01", "2020-10-12"]),
            axis=alt.Axis(tickCount=8, values=list(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")['Period'].values))),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f'))
    ).properties(title={'text': 'CEWS evolution', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## Rural/urban comparison

In [25]:
cews.query("GeoAggregation == 'urban/rural by country' and IndustryAggregation == 'All industries'").head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
71157,2020-03-15,rural,Canada,RURAL,Not applicable,TOTAL,All Industries,36135.0,6.886580e+08,326654.0,3380.0,urban/rural by country,All industries,NaN,NaN,NaN
71265,2020-03-15,urban,Canada,URBAN,Not applicable,TOTAL,All Industries,257935.0,6.771338e+09,3162560.0,27140.0,urban/rural by country,All industries,NaN,NaN,NaN
148265,2020-04-12,rural,Canada,RURAL,Not applicable,TOTAL,All Industries,43440.0,8.902310e+08,382773.0,4350.0,urban/rural by country,All industries,NaN,NaN,NaN


In [95]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and IndustryAggregation == 'All industries'")).mark_bar(size=40).encode(
    x=alt.X('Period:T', title='Date', scale=alt.Scale(domain=["2020-03-01", "2020-10-12"]),
            axis=alt.Axis(tickCount=8, values=list(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")['Period'].values))),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f')),
    color=alt.Color('GeographicClassification:N', legend=alt.Legend(title="Greographic classification", orient="top-right")),
    column=alt.Column('site:N', header=alt.Header(title=None, labelOrient='bottom'))
    ).properties(title={'text': 'CEWS rural/urban breakdown evolution', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [119]:
# Remake the above for Scotts notes

temp = cews.query("GeoAggregation == 'urban/rural by country' and IndustryAggregation == 'All industries'")

alt.Chart(temp, title = "Total CEWS Subsidies Given In Each Claim Period").mark_bar().encode(
    x = alt.X("GeographicClassification", title = None, axis = alt.Axis(labels = False)),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
        axis=alt.Axis(format='$,f')),
    color = alt.Color("GeographicClassification", title = None),
    column = "Period"
).configure_axis(domain=False, grid=False, ticks=False).configure_view(stroke = "transparent")

alt.Chart(...)

In [27]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and IndustryAggregation == 'All industries'")).mark_bar().encode(
    y=alt.Y('GeographicClassification:N', title=None),
    x=alt.X('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(format='$,f')),
    color=alt.Color('GeographicClassification:N', legend=None),
    ).properties(title={'text': 'CEWS rural/urban breakdown comparison', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## Rural totals

### CEWS rural evolution and average number of rehired employees and supported employees and businesses by claim period

In [28]:
cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'All industries'").head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
71157,2020-03-15,rural,Canada,RURAL,Not applicable,TOTAL,All Industries,36135.0,6.886580e+08,326654.0,3380.0,urban/rural by country,All industries,NaN,NaN,NaN
148265,2020-04-12,rural,Canada,RURAL,Not applicable,TOTAL,All Industries,43440.0,8.902310e+08,382773.0,4350.0,urban/rural by country,All industries,NaN,NaN,NaN
225225,2020-05-10,rural,Canada,RURAL,Not applicable,TOTAL,All Industries,43910.0,1.016442e+09,432676.0,5340.0,urban/rural by country,All industries,NaN,NaN,NaN


In [29]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('sum(Subsidy):Q', format=('$,.0f'))
        ).properties(title={'text': 'CEWS total dollar value of subsidies approved', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [30]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(Subsidy):Q', format=('$,.0f'))
        ).properties(title={'text': 'CEWS average dollar value of subsidies approved', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [31]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(SupportedEmployees):Q', format=(',.0f'))
        ).properties(title={'text': 'CEWS average supported employees', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [32]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(RehiredEmployees):Q', format=(',.0f'))
        ).properties(title={'text': 'CEWS average rehired employees', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [33]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'All industries'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(BusinessLocations):Q', format=(',.0f'))
        ).properties(title={'text': 'CEWS average businesses supported', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [34]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'All industries'")).mark_bar(size=40).encode(
    x=alt.X('Period:T', title='Date', scale=alt.Scale(domain=["2020-03-01", "2020-10-12"]),
            axis=alt.Axis(tickCount=8, values=list(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")['Period'].values))),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f'))
    ).properties(title={'text': 'CEWS evolution', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## Rural industry breakdown 

In [35]:
cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'Level 1'").head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
71051,2020-03-15,rural,Canada,RURAL,Not applicable,11,"Agriculture, forestry, fishing and hunting",2150.0,NaN,NaN,85.0,urban/rural by country,Level 1,NaN,NaN,NaN
71057,2020-03-15,rural,Canada,RURAL,Not applicable,21,"Mining, quarrying, and oil and gas extraction",785.0,62895000.0,20647.0,25.0,urban/rural by country,Level 1,NaN,NaN,NaN
71061,2020-03-15,rural,Canada,RURAL,Not applicable,22,Utilities,25.0,NaN,NaN,0.0,urban/rural by country,Level 1,NaN,NaN,NaN


In [36]:
temp = cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'Level 1'")
inds = list(np.unique(temp["Industry"]))

alt.Chart(temp).mark_bar().encode(
    color = alt.Color("Industry", scale = alt.Scale(domain = inds, range = getIndCols(inds)), legend = None),
    y=alt.Y('Industry:N', sort='-x',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('Subsidy', type='quantitative', aggregate='sum', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f'))
    ).properties(title={'text': 'CEWS total dollar value of subsidies approved to rural areas by industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [37]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'Level 1'")).mark_bar().encode(
    y=alt.Y('Industry:N', sort='-x',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('Subsidy', type='quantitative', aggregate='average', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f'))
    ).properties(title={'text': 'CEWS average dollar value of subsidies approved to rural areas by industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [38]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'Level 1'")).mark_bar().encode(
    y=alt.Y('Industry:N', sort='-x',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('SupportedEmployees', type='quantitative', aggregate='average', title='Supported employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format=',f'))
    ).properties(title={'text': 'CEWS average supported employees in rural areas by industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [39]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'Level 1'")).mark_bar().encode(
    y=alt.Y('Industry:N', sort='-x',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('RehiredEmployees', type='quantitative', aggregate='average', title='Supported employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format=',f'))
    ).properties(title={'text': 'CEWS average rehired employees in rural areas by industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [40]:
alt.Chart(cews.query("GeoAggregation == 'urban/rural by country' and GeographicClassification == 'RURAL' and IndustryAggregation == 'Level 1'")).mark_bar().encode(
    y=alt.Y('Industry:N', sort='-x',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('BusinessLocations', type='quantitative', aggregate='average', title='Businesses supported',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format=',f'))
    ).properties(title={'text': 'CEWS average businesses supported in rural areas by industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## CEWS top three supported rural industries

In [41]:
industries = ['Construction', 'Manufacturing', 'Accommodation and food services', 'Mining, quarrying, and oil and gas extraction']

df = cews.query("GeoAggregation == 'urban/rural by province' and GeographicClassification == 'RURAL'"
          )[cews.query("GeoAggregation == 'urban/rural by province' and GeographicClassification == 'RURAL'"
                      ).RegionCode.isin(provinces)]

top_ind = (df[df.Industry.isin(industries)]
          ).append(((cews.query("Region == 'Canada' and GeographicClassification == 'RURAL' and Industry == 'Construction'")
                    ).append(cews.query("Region == 'Canada' and GeographicClassification == 'RURAL' and Industry == 'Manufacturing'"))
                    ).append(cews.query("Region == 'Canada' and GeographicClassification == 'RURAL' and Industry == 'Accommodation and food services'"))
                    ).append(cews.query("Region == 'Canada' and GeographicClassification == 'RURAL' and Industry == 'Mining, quarrying, and oil and gas extraction'"))

dummy = top_ind.Region.str.split('-', n=1, expand=True)
top_ind.Region = dummy[0].str.rstrip()

df2 = cews.query("GeoAggregation == 'urban/rural by province' and GeographicClassification == 'URBAN'"
          )[cews.query("GeoAggregation == 'urban/rural by province' and GeographicClassification == 'URBAN'"
                      ).RegionCode.isin(provinces_ur)]

top_ind_ur = (df2[df2.Industry.isin(industries)]
          ).append(((cews.query("Region == 'Canada' and GeographicClassification == 'URBAN' and Industry == 'Construction'")
                    ).append(cews.query("Region == 'Canada' and GeographicClassification == 'URBAN' and Industry == 'Manufacturing'"))
                    ).append(cews.query("Region == 'Canada' and GeographicClassification == 'URBAN' and Industry == 'Accommodation and food services'"))
                    ).append(cews.query("Region == 'Canada' and GeographicClassification == 'RURAL' and Industry == 'Mining, quarrying, and oil and gas extraction'"))

dummy = top_ind_ur.Region.str.split('-', n=1, expand=True)
top_ind_ur.Region = dummy[0].str.rstrip()

top_ind.head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
104,2020-03-15,10-rural,Newfoundland and Labrador,RURAL,Not applicable,21,"Mining, quarrying, and oil and gas extraction",10.0,NaN,NaN,0.0,urban/rural by province,Level 1,10.0,Newfoundland and Labrador,NaN
108,2020-03-15,10-rural,Newfoundland and Labrador,RURAL,Not applicable,23,Construction,110.0,1998000.0,789.0,5.0,urban/rural by province,Level 1,10.0,Newfoundland and Labrador,NaN
112,2020-03-15,10-rural,Newfoundland and Labrador,RURAL,Not applicable,31-33,Manufacturing,70.0,3536000.0,1408.0,10.0,urban/rural by province,Level 1,10.0,Newfoundland and Labrador,NaN


In [42]:
# Creating a dataframe with the total number of employees 
re_counts.rename(columns={'GEO': 'Region', 'North American Industry Classification System (NAICS)' : 'Industry', 'VALUE' : 'Total Employees'}, inplace=True)
dummy = re_counts.Industry.str.split('[', n=1, expand=True)
re_counts['IndustryCode'] = dummy[1].str.split(']').str[0]
re_counts['Total Employees'] = re_counts['Total Employees'] * 1000
re_counts.Industry = dummy[0].str.rstrip()

ue_counts.rename(columns={'GEO': 'Region', 'North American Industry Classification System (NAICS)' : 'Industry', 'VALUE' : 'Total Employees'}, inplace=True)
dummy = ue_counts.Industry.str.split('[', n=1, expand=True)
ue_counts['IndustryCode'] = dummy[1].str.split(']').str[0]
ue_counts['Total Employees'] = ue_counts['Total Employees'] * 1000
ue_counts.Industry = dummy[0].str.rstrip()

## Average values
avg_top_ind = top_ind.groupby(['Region', 'Industry', 'GeographicClassification']).mean().reset_index()

avg_top_ind_ur = top_ind_ur.groupby(['Region', 'Industry', 'GeographicClassification']).mean().reset_index()


# Merging the dataframes
avg_top_ind = pd.merge(avg_top_ind, re_counts,  how='right', on=['Region','Industry'])

avg_top_ind_ur = pd.merge(avg_top_ind_ur, ue_counts,  how='right', on=['Region','Industry'])

# Aopending the dataframes
avg_top_ind = avg_top_ind.append(avg_top_ind_ur)

#Calculating the ratio of total supported businesses
avg_top_ind['Employees Ratio'] = avg_top_ind['SupportedEmployees'] / avg_top_ind['Total Employees']

avg_top_ind.head(3)

,Region,Industry,GeographicClassification,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,ProvinceCode,TotalEmployees,Total Employees,IndustryCode,Employees Ratio
0,Canada,Construction,RURAL,4997.500,95475250.0,45032.125,350.000,NaN,NaN,239400.0,23,0.188104
1,Canada,Manufacturing,RURAL,2502.500,166161875.0,83344.125,194.375,NaN,NaN,266200.0,31-33,0.313088
2,Canada,Accommodation and food services,RURAL,4641.875,70215500.0,65316.625,640.000,NaN,NaN,119800.0,72,0.545214


In [43]:
alt.Chart(top_ind.query("Region == 'Canada'")).mark_bar(size=40).encode(
    x=alt.X('Period:T', title='Date', scale=alt.Scale(domain=["2020-03-01", "2020-10-12"]),
            axis=alt.Axis(tickCount=8, values=list(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")['Period'].values))),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f')),
    color=alt.Column('Industry:N', legend=alt.Legend(title="Industry", orient="top"), scale = alt.Scale(domain = industries, range = getIndCols(industries))),
    column=alt.Column('Industry:N', title=None),
    ).properties(title={'text': 'CEWS most supported rural industries evolution', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [44]:
fig1 = alt.Chart(top_ind.query("Region == 'Alberta'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('Subsidy', type='quantitative', aggregate='sum',
            title='Dollar value of subsidies approved', axis=alt.Axis(format='$,f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'Total CEWS Subsidies (Rural Alberta)'}
                )

In [45]:
fig2 = alt.Chart(top_ind.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('Subsidy', type='quantitative', aggregate='sum',
            title='Dollar value of subsidies approved', axis=alt.Axis(format='$,f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'Total CEWS Subsidies (Rural Canada)'}
                )

(fig1 & fig2).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.VConcatChart(...)

In [46]:
alt.Chart(top_ind.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('Subsidy', type='quantitative', aggregate='average',
            title='Dollar value of subsidies approved', axis=alt.Axis(format='$,f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'CEWS average dollar value of subsidies approved', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [47]:
alt.Chart(top_ind.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('SupportedEmployees', type='quantitative', aggregate='average',
            title='Supported employees', axis=alt.Axis(format=',f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'CEWS average supported employees', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [48]:
alt.Chart(top_ind.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('RehiredEmployees', type='quantitative', aggregate='average',
            title='Rehired employees', axis=alt.Axis(format=',f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'CEWS average rehired employees', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [49]:
alt.Chart(top_ind.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('BusinessLocations', type='quantitative', aggregate='average',
            title='Businesses supported', axis=alt.Axis(format=',f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'CEWS average businesses supported', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [50]:
alt.Chart(avg_top_ind.query("Region == 'Canada' and GeographicClassification =='RURAL'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('Employees Ratio', type='quantitative', scale=alt.Scale(domain=(0,1)),
            title='Proportion of employees supported', axis=alt.Axis(format='%')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'CEWS average proportion of employees supported', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## CEWS accomodation and food services industry rural/urban comparison

In [51]:
acco_food = cews[cews.IndustryCode.str.startswith('72')].query("GeoAggregation == 'urban/rural by province' and IndustryAggregation == 'Level 1'"
                                ).append(cews[cews.IndustryCode.str.startswith('72')].query("GeoAggregation == 'urban/rural by country' and IndustryAggregation == 'Level 1'"))
dummy = acco_food.Region.str.split('-', n=1, expand=True)
acco_food.Region = dummy[0].str.rstrip()

acco_food.head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
184,2020-03-15,10-rural,Newfoundland and Labrador,RURAL,Not applicable,72,Accommodation and food services,170.0,1822000.0,1558.0,10.0,urban/rural by province,Level 1,10.0,Newfoundland and Labrador,NaN
279,2020-03-15,10-urban,Newfoundland and Labrador,URBAN,Not applicable,72,Accommodation and food services,300.0,5937000.0,4999.0,40.0,urban/rural by province,Level 1,10.0,Newfoundland and Labrador,NaN
2714,2020-03-15,11-rural,Prince Edward Island,RURAL,Not applicable,72,Accommodation and food services,40.0,369000.0,438.0,0.0,urban/rural by province,Level 1,11.0,Prince Edward Island,NaN


In [52]:
alt.Chart(acco_food.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('GeographicClassification:N', title=None, sort='x'),
    x=alt.X('Subsidy', type='quantitative', aggregate='sum',
            title='Total dollar value of subsidies approved', axis=alt.Axis(format='$,f')),
    color=alt.Color('GeographicClassification:N', legend=None),
    ).properties(title={'text': 'Accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [53]:
alt.Chart(acco_food.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('GeographicClassification:N', title=None, sort='x'),
    x=alt.X('Subsidy', type='quantitative', aggregate='average',
            title='Average dollar value of subsidies approved', axis=alt.Axis(format='$,f')),
    color=alt.Color('GeographicClassification:N', legend=None),
    ).properties(title={'text': 'Accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [54]:
alt.Chart(acco_food.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('GeographicClassification:N', title=None, sort='x'),
    x=alt.X('SupportedEmployees', type='quantitative', aggregate='average',
            title='Average supported employees', axis=alt.Axis(format=',f')),
    color=alt.Color('GeographicClassification:N', legend=None),
    ).properties(title={'text': 'Accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [55]:
alt.Chart(acco_food.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('GeographicClassification:N', title=None, sort='x'),
    x=alt.X('RehiredEmployees', type='quantitative', aggregate='average',
            title='Average rehired employees', axis=alt.Axis(format=',f')),
    color=alt.Color('GeographicClassification:N', legend=None),
    ).properties(title={'text': 'Accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [56]:
alt.Chart(acco_food.query("Region == 'Canada'")).mark_bar().encode(
    y=alt.Y('GeographicClassification:N', title=None, sort='x'),
    x=alt.X('BusinessLocations', type='quantitative', aggregate='average',
            title='Average businesses supported', axis=alt.Axis(format=',f')),
    color=alt.Color('GeographicClassification:N', legend=None),
    ).properties(title={'text': 'Accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [57]:
alt.Chart(acco_food.query("Region == 'Canada'")).mark_bar(size=40).encode(
    x=alt.X('Period:T', title='Date', scale=alt.Scale(domain=["2020-03-01", "2020-10-12"]),
            axis=alt.Axis(tickCount=8, values=list(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")['Period'].values))),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f')),
    color=alt.Column('GeographicClassification:N', legend=alt.Legend(title="Geographic classification", orient="top")),
    column=alt.Column('GeographicClassification:N', title=None),
    ).properties(title={'text': 'Accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [58]:
temp = avg_top_ind.query("Industry == 'Accommodation and food services'")

provs = sorted(list(cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'All industries'")["Province"].unique())) + ["Canada"]

alt.Chart(temp).mark_bar().encode(
    x=alt.X('Region:N', title=None, sort='-y',
           axis=alt.Axis(labels=False)),
    y=alt.Y('Employees Ratio', type='quantitative', scale=alt.Scale(domain=(0,1)),
            title='CEWS average proportion of employees supported', axis=alt.Axis(format='%')),
    color=alt.Color('Region:N', legend=alt.Legend(title="Province", orient="bottom", columns=4), scale = alt.Scale(domain = provs, range = getProvCols(provs))),
    column=alt.Column('GeographicClassification:N', title='Geographic classification')
    ).properties(title={'text': 'Accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [59]:
temp = avg_top_ind.query("Industry == 'Construction'")

provs = sorted(list(cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'All industries'")["Province"].unique())) + ["Canada"]

alt.Chart(temp).mark_bar().encode(
    x=alt.X('Region:N', title=None, sort='-y',
           axis=alt.Axis(labels=False)),
    y=alt.Y('Employees Ratio', type='quantitative', scale=alt.Scale(domain=(0,1)),
            title='CEWS average proportion of employees supported', axis=alt.Axis(format='%')),
    color=alt.Color('Region:N', legend=alt.Legend(title="Province", orient="bottom", columns=4), scale = alt.Scale(domain = provs, range = getProvCols(provs))),
    column=alt.Column('GeographicClassification:N', title='Geographic classification')
    ).properties(title={'text': 'Construction industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [60]:
industries

['Construction',
 'Manufacturing',
 'Accommodation and food services',
 'Mining, quarrying, and oil and gas extraction']

In [61]:
temp = avg_top_ind.query("Industry == 'Manufacturing'")

provs = sorted(list(cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'All industries'")["Province"].unique())) + ["Canada"]

alt.Chart(temp).mark_bar().encode(
    x=alt.X('Region:N', title=None, sort='-y',
           axis=alt.Axis(labels=False)),
    y=alt.Y('Employees Ratio', type='quantitative', scale=alt.Scale(domain=(0,1)),
            title='CEWS average proportion of employees supported', axis=alt.Axis(format='%')),
    color=alt.Color('Region:N', legend=alt.Legend(title="Province", orient="bottom", columns=4), scale = alt.Scale(domain = provs, range = getProvCols(provs))),
    column=alt.Column('GeographicClassification:N', title='Geographic classification')
    ).properties(title={'text': 'Manufacturing industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [62]:
temp = avg_top_ind.query("Industry == 'Mining, quarrying, and oil and gas extraction'")

provs = sorted(list(cews.query("GeoAggregation == 'Province' and IndustryAggregation == 'All industries'")["Province"].unique())) + ["Canada"]

alt.Chart(temp).mark_bar().encode(
    x=alt.X('Region:N', title=None, sort='-y',
           axis=alt.Axis(labels=False)),
    y=alt.Y('Employees Ratio', type='quantitative', scale=alt.Scale(domain=(0,1)),
            title='CEWS average proportion of employees supported', axis=alt.Axis(format='%')),
    color=alt.Color('Region:N', legend=alt.Legend(title="Province", orient="bottom", columns=4), scale = alt.Scale(domain = provs, range = getProvCols(provs))),
    column=alt.Column('GeographicClassification:N', title='Geographic classification')
    ).properties(title={'text': 'Manufacturing industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [63]:
avg_top_ind.query("Industry == 'Mining, quarrying, and oil and gas extraction'")


,Region,Industry,GeographicClassification,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,ProvinceCode,TotalEmployees,Total Employees,IndustryCode,Employees Ratio


## CEWS accomodation and food services industry in rural Alberta

In [64]:
alberta_acco_food = cews[cews.IndustryCode.str.startswith('72')]
alberta_acco_food = alberta_acco_food[alberta_acco_food.RegionCode.str.startswith('48')]

alberta_acco_food.head(3)

,Period,RegionCode,Region,GeographicClassification,CensusLevel,IndustryCode,Industry,BusinessLocations,Subsidy,SupportedEmployees,RehiredEmployees,GeoAggregation,IndustryAggregation,ProvinceCode,Province,TotalEmployees
52412,2020-03-15,48,Alberta,Not applicable,Not applicable,72,Accommodation and food services,4470.0,80660000.0,66393.0,695.0,Province,Level 1,48.0,Alberta,141335.0
52413,2020-03-15,48,Alberta,Not applicable,Not applicable,721,Accommodation services,640.0,20364000.0,13530.0,70.0,Province,Level 2,48.0,Alberta,NaN
52414,2020-03-15,48,Alberta,Not applicable,Not applicable,722,Food services and drinking places,3830.0,60296000.0,52864.0,625.0,Province,Level 2,48.0,Alberta,NaN


In [65]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 1'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('sum(Subsidy):Q', format=('$,.0f'))
        ).properties(title={'text': 'CEWS total dollar value of subsidies approved', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [66]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 1'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(Subsidy):Q', format=('$,.0f'))
        ).properties(title={'text': 'CEWS average dollar value of subsidies approved', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [67]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 1'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(SupportedEmployees):Q', format=(',.0f'))
        ).properties(title={'text': 'CEWS average supported employees', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [68]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 1'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(RehiredEmployees):Q', format=(',.0f'))
        ).properties(title={'text': 'CEWS average rehired employees', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [69]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 1'")).mark_text(color='darkblue', size=40).encode(
        text=alt.Text('average(BusinessLocations):Q', format=(',.0f'))
        ).properties(title={'text': 'CEWS average businesses supported', 'subtitle': 'First eight four-week claim period'}
                    ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [70]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 1'")).mark_bar(size=40).encode(
    x=alt.X('Period:T', title='Date', scale=alt.Scale(domain=["2020-03-01", "2020-10-12"]),
            axis=alt.Axis(tickCount=8, values=list(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")['Period'].values))),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f'))
    ).properties(title={'text': 'CEWS evolution', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## CEWS accomodation and food services industry in rural Alberta breakdown

In [71]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 2'")).mark_bar(size=40).encode(
    x=alt.X('Period:T', title='Date', scale=alt.Scale(domain=["2020-03-01", "2020-10-12"]),
            axis=alt.Axis(tickCount=8, values=list(cews.query("Region == 'Canada' and RegionCode == 'TOTAL' and IndustryAggregation == 'All industries'")['Period'].values))),
    y=alt.Y('Subsidy', type='quantitative', title='Dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10, format='$,f')),
    color=alt.Column('Industry:N', legend=alt.Legend(title="Industry sub-sector", orient="top")),
    column=alt.Column('Industry:N', title=None),
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [72]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 2'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='x'),
    x=alt.X('Subsidy', type='quantitative', aggregate='sum',
            title='Total dollar value of subsidies approved', axis=alt.Axis(format='$,f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [73]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 2'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='x'),
    x=alt.X('Subsidy', type='quantitative', aggregate='average',
            title='Average dollar value of subsidies approved', axis=alt.Axis(format='$,f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [74]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 2'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='x'),
    x=alt.X('SupportedEmployees', type='quantitative', aggregate='average',
            title='Average supported employees', axis=alt.Axis(format=',f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [75]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 2'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('RehiredEmployees', type='quantitative', aggregate='average',
            title='Average rehired employees', axis=alt.Axis(format=',f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [76]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation =='urban/rural by province' and IndustryAggregation == 'Level 2'")).mark_bar().encode(
    y=alt.Y('Industry:N', title=None, sort='-x'),
    x=alt.X('BusinessLocations', type='quantitative', aggregate='average',
            title='Average businesses supported', axis=alt.Axis(format=',f')),
    color=alt.Color('Industry:N', legend=None),
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## Most supported CSD in the accomodation and food services industry in rural Alberta

In [77]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 1'"
                       ).groupby('Region').mean().sort_values('Subsidy', ascending=False).head(10).reset_index()).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('Subsidy', type='quantitative', aggregate='average', title='Average dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format='$,f'))
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [78]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 1'"
                       ).groupby('Region').mean().sort_values('SupportedEmployees', ascending=False).head(10).reset_index()).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('SupportedEmployees', type='quantitative', aggregate='average', title='Average supported employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [79]:
alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 1'"
                       ).groupby('Region').mean().sort_values('RehiredEmployees', ascending=False).head(10).reset_index()).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('RehiredEmployees', type='quantitative', aggregate='average', title='Average rehired employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

alt.Chart(alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 1'"
                       ).groupby('Region').mean().sort_values('BusinessLocations', ascending=False).head(10).reset_index()).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('BusinessLocations', type='quantitative', aggregate='average', title='Average supported businesses',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta accommodation and food services industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

## Most supported CSD in the accomodation and food services industry in rural Alberta

In [80]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Accommodation services'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('Subsidy', type='quantitative', aggregate='average', title='Average dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format='$,f'))
    ).properties(title={'text': 'Rural Alberta accommodation services subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [81]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Accommodation services'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('SupportedEmployees', type='quantitative', aggregate='average', title='Average supported employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta accommodation services subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [82]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Accommodation services'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('RehiredEmployees', type='quantitative', aggregate='average', title='Average rehired employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta accommodation services subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [83]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Accommodation services'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('BusinessLocations', type='quantitative', aggregate='average', title='Average supported businesses',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta accommodation services subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [84]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Food services and drinking places'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('Subsidy', type='quantitative', aggregate='average', title='Average dollar value of subsidies approved',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format='$,f'))
    ).properties(title={'text': 'Rural Alberta food services and drinking places subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [85]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Food services and drinking places'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('SupportedEmployees', type='quantitative', aggregate='average', title='Average supported employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta food services and drinking places subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [86]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Food services and drinking places'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('RehiredEmployees', type='quantitative', aggregate='average', title='Average rehired employees',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta food services and drinking places subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

In [87]:
alt.Chart((alberta_acco_food.query("GeographicClassification == 'RURAL' and GeoAggregation != 'urban/rural by province' and IndustryAggregation == 'Level 2'"
                       ).groupby(['Region', 'Industry']).mean().sort_values('Subsidy', ascending=False).reset_index()
                        ).query("Industry == 'Food services and drinking places'").sort_values('Subsidy', ascending=False).head(5)).mark_bar().encode(
    y=alt.Y('Region:N', sort='-x', title='Most supported CSD',
           axis=alt.Axis(titleAngle=0, titleAnchor='start', titleY=-10)),
    x=alt.X('BusinessLocations', type='quantitative', aggregate='average', title='Average supported businesses',
            axis=alt.Axis(titleAngle=0, titleAnchor='start', format=',F'))
    ).properties(title={'text': 'Rural Alberta food services and drinking places subsector industry', 'subtitle': 'First eight four-week claim period'}
                ).configure_axis(domain=False, grid=False, ticks=False
                                ).configure_view(stroke="transparent")

alt.Chart(...)

## Most subsidized CSDs of each type in Rural Alberta

In [88]:
inds

['Accommodation and food services',
 'Administrative and support, waste management and remediation services',
 'Agriculture, forestry, fishing and hunting',
 'Arts, entertainment and recreation',
 'Construction',
 'Educational services',
 'Finance and insurance',
 'Health care and social assistance',
 'Information and cultural industries',
 'Management of companies and enterprises',
 'Manufacturing',
 'Mining, quarrying, and oil and gas extraction',
 'Other and Missing NAICS',
 'Professional, scientific and technical services',
 'Real estate and rental and leasing',
 'Retail trade',
 'Transportation and warehousing',
 'Utilities',
 'Wholesale trade']

In [89]:
temp = cews.query("GeoAggregation == 'CSD' and GeographicClassification == 'RURAL' and Industry == 'Accommodation and food services' and Province == 'Alberta'")
temp = temp.groupby(["Region"]).sum().reset_index().sort_values("Subsidy")
fig1 = alt.Chart(temp.tail(5), title = "Accommodation and Food Services").mark_bar(color = getIndCols(["Accommodation and food services"])[0]).encode(
    x = alt.X("Subsidy"),
    y = alt.Y("Region", sort = "-x"),
)

In [90]:
temp = cews.query("GeoAggregation == 'CSD' and GeographicClassification == 'RURAL' and Industry == 'Mining, quarrying, and oil and gas extraction' and Province == 'Alberta'")
temp = temp.groupby(["Region"]).sum().reset_index().sort_values("Subsidy")
fig2= alt.Chart(temp.tail(5), title = "Mining, quarrying, and oil and gas extraction").mark_bar(color = getIndCols(["Mining, quarrying, and oil and gas extraction"])[0]).encode(
    x = alt.X("Subsidy"),
    y = alt.Y("Region", sort = "-x"),
)

In [91]:
temp = cews.query("GeoAggregation == 'CSD' and GeographicClassification == 'RURAL' and Industry == 'Manufacturing' and Province == 'Alberta'")
temp = temp.groupby(["Region"]).sum().reset_index().sort_values("Subsidy")
fig3 = alt.Chart(temp.tail(5), title = "Manufacturing").mark_bar(color = getIndCols(["Manufacturing"])[0]).encode(
    x = alt.X("Subsidy"),
    y = alt.Y("Region", sort = "-x"),
)

In [92]:
temp = cews.query("GeoAggregation == 'CSD' and GeographicClassification == 'RURAL' and Industry == 'Construction' and Province == 'Alberta'")
temp = temp.groupby(["Region"]).sum().reset_index().sort_values("Subsidy")
fig4 = alt.Chart(temp.tail(5), title = "Construction").mark_bar(color = getIndCols(["Construction"])[0]).encode(
    x = alt.X("Subsidy"),
    y = alt.Y("Region", sort = "-x"),
)


In [93]:
((fig1 | fig2) & (fig3 | fig4)).properties(title = "The Four Most Subsidized Industries in Rural Alberta, and the Five CSDs Recieving the Largest CEWS Subsidy in Each")

alt.VConcatChart(...)